## Document about the schools dataset, done by Patricia Correia

In [272]:
import pandas as pd


In [273]:
df = pd.read_csv('Data/primary_schools.csv')


In [274]:
df.head(5)

,Roll Number,Official Name,Address (Line 1),Address (Line 2),Address (Line 3),Address (Line 4),County Description,Local Authority Description,Phone No.,Principal Name,Email,Eircode,Gaeltacht Indicator (Y/N),DEIS (Y/N),Island (Y/N),Irish Classification Description,Ethos Description,Female,Male,Total
0,00697S,ST BRIGIDS MXD N S,Beechpark Lawn,Castleknock,Dublin 15,NaN,Dublin,Fingal County Council,18214040,Denis Courtney,principal@saintbrigids.ie,D15P820,N,N,N,No subjects through Irish,Catholic,447,502.0,949
1,00714P,LUCAN B N S,Chapel Hill,Lucan,Dublin,NaN,Dublin,South Dublin County Council,16281857,Dara Burke,info@stmarysbnslucan.com,K78YD27,N,N,N,No subjects through Irish,Catholic,0,513.0,513
2,00729F,CLOCHAR LORETO N S,Grange Road,Rathfarnham,Dublin 14,NaN,Dublin,South Dublin County Council,14931640,Sr. Maria Hyland,info@loretograngeroad.ie,D14YY28,N,N,N,No subjects through Irish,Catholic,501,0.0,501
3,00752A,CENTRAL SENIOR MXD N S,Marlborough Street,City Centre,Dublin1,NaN,Dublin,Dublin City Council,18788103,Deirdre Gartland,modesenmix@eircom.net,D01P027,N,Y,N,No subjects through Irish,Catholic,133,143.0,276
4,01170G,S N NA H-AILLE,Westown,Naul,Dublin,NaN,Dublin,Fingal County Council,18413821,Edel McMahon,naulns@eircom.net,K32PX40,N,N,N,No subjects through Irish,Catholic,67,72.0,139


In [275]:
df.shape

(452, 20)

## Rename columns 

In [276]:
df_new = df.rename({'Address (Line 1)':'address1', 'Address (Line 2)':'address2', 'Address (Line 3)':'address3', 'Address (Line 4)':'address4', 'County Description':'city', 'Official Name': 'schoolName'}, axis='columns')

In [277]:
df_new.head()

,Roll Number,schoolName,address1,address2,address3,address4,city,Local Authority Description,Phone No.,Principal Name,Email,Eircode,Gaeltacht Indicator (Y/N),DEIS (Y/N),Island (Y/N),Irish Classification Description,Ethos Description,Female,Male,Total
0,00697S,ST BRIGIDS MXD N S,Beechpark Lawn,Castleknock,Dublin 15,NaN,Dublin,Fingal County Council,18214040,Denis Courtney,principal@saintbrigids.ie,D15P820,N,N,N,No subjects through Irish,Catholic,447,502.0,949
1,00714P,LUCAN B N S,Chapel Hill,Lucan,Dublin,NaN,Dublin,South Dublin County Council,16281857,Dara Burke,info@stmarysbnslucan.com,K78YD27,N,N,N,No subjects through Irish,Catholic,0,513.0,513
2,00729F,CLOCHAR LORETO N S,Grange Road,Rathfarnham,Dublin 14,NaN,Dublin,South Dublin County Council,14931640,Sr. Maria Hyland,info@loretograngeroad.ie,D14YY28,N,N,N,No subjects through Irish,Catholic,501,0.0,501
3,00752A,CENTRAL SENIOR MXD N S,Marlborough Street,City Centre,Dublin1,NaN,Dublin,Dublin City Council,18788103,Deirdre Gartland,modesenmix@eircom.net,D01P027,N,Y,N,No subjects through Irish,Catholic,133,143.0,276
4,01170G,S N NA H-AILLE,Westown,Naul,Dublin,NaN,Dublin,Fingal County Council,18413821,Edel McMahon,naulns@eircom.net,K32PX40,N,N,N,No subjects through Irish,Catholic,67,72.0,139


In [278]:
df_new.shape

(452, 20)

In [279]:
#pip install geopy

In [280]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
import time
from geopy.extra.rate_limiter import RateLimiter

## * Create locator that holds the Geocoding service, Photon.

In [281]:
locator = Photon(user_agent="measurements")

## * Concatenate address columns into one that is appropriate for geocoding

In [282]:
df_new['full_address'] = df_new.address1 + "," + df_new.address2 + "," + df_new.city

In [283]:
#geocode
df_new['gcode'] = df_new.full_address.apply(locator.geocode)


## — Create a df['location'] column by applying geocode we created.
## — Third, we can create latitude, longitude, and altitude as a single tuple column.
## — Finally, We split latitude, longitude, and altitude columns into three separate columns.

In [284]:
 #1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df_new['location'] = df_new['full_address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df_new['point'] = df_new['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df_new[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_new['point'].tolist(), index=df_new.index)


RateLimiter caught an error, retrying (0/2 tries). Called with (*('North Brunswick Street,Dublin 7,Dublin',), **{}).
Traceback (most recent call last):
  File "C:\Users\atricia\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\atricia\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\atricia\anaconda3\lib\http\client.py", line 1332, in getresponse
    response.begin()
  File "C:\Users\atricia\anaconda3\lib\http\client.py", line 303, in begin
    version, status, reason = self._read_status()
  File "C:\Users\atricia\anaconda3\lib\http\client.py", line 264, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\atricia\anaconda3\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\atricia\anaconda3\lib\ssl

In [285]:
df_new.head()

,Roll Number,schoolName,address1,address2,address3,address4,city,Local Authority Description,Phone No.,Principal Name,...,Female,Male,Total,full_address,gcode,location,point,latitude,longitude,altitude
0,00697S,ST BRIGIDS MXD N S,Beechpark Lawn,Castleknock,Dublin 15,NaN,Dublin,Fingal County Council,18214040,Denis Courtney,...,447,502.0,949,"Beechpark Lawn,Castleknock,Dublin","(Beechpark Lawn, D15V9KN, Castleknock, Éire / ...","(Beechpark Lawn, D15V9KN, Castleknock, Éire / ...","(53.375855, -6.361625, 0.0)",53.375855,-6.361625,0.0
1,00714P,LUCAN B N S,Chapel Hill,Lucan,Dublin,NaN,Dublin,South Dublin County Council,16281857,Dara Burke,...,0,513.0,513,"Chapel Hill,Lucan,Dublin","(Chapel Hill, K78 N8X6, Lucan, Éire / Ireland,...","(Chapel Hill, K78 N8X6, Lucan, Éire / Ireland,...","(53.360019, -6.4388757, 0.0)",53.360019,-6.438876,0.0
2,00729F,CLOCHAR LORETO N S,Grange Road,Rathfarnham,Dublin 14,NaN,Dublin,South Dublin County Council,14931640,Sr. Maria Hyland,...,501,0.0,501,"Grange Road,Rathfarnham,Dublin","(Grange Road, DUBLIN 14, Rathfarnham, Éire / I...","(Grange Road, DUBLIN 14, Rathfarnham, Éire / I...","(53.2953171, -6.2825391, 0.0)",53.295317,-6.282539,0.0
3,00752A,CENTRAL SENIOR MXD N S,Marlborough Street,City Centre,Dublin1,NaN,Dublin,Dublin City Council,18788103,Deirdre Gartland,...,133,143.0,276,"Marlborough Street,City Centre,Dublin","(LRG, Marlborough Street, D01P7K8, Marlborough...","(LRG, Marlborough Street, D01P7K8, Marlborough...","(53.3493141, -6.2576704, 0.0)",53.349314,-6.257670,0.0
4,01170G,S N NA H-AILLE,Westown,Naul,Dublin,NaN,Dublin,Fingal County Council,18413821,Edel McMahon,...,67,72.0,139,"Westown,Naul,Dublin","(Westown, Naul, Éire / Ireland, (53.5792861000...","(Westown, Naul, Éire / Ireland, (53.5792861000...","(53.579286100000004, -6.299280855181822, 0.0)",53.579286,-6.299281,0.0


In [286]:
# delete columns that are not relevant
df_new = df_new.drop(['address1', 'address2', 'address3', 'address4', 'full_address', 'altitude', 'city', 'Local Authority Description', 'gcode', 'location', 'point', 'Gaeltacht Indicator (Y/N)', 'DEIS (Y/N)', 'Island (Y/N)', 'Irish Classification Description', 'Eircode'], axis =1)

In [287]:
df_new['Type'] = 'Primary School'

In [288]:
df_new.shape

(452, 12)

In [289]:
df_new

,Roll Number,schoolName,Phone No.,Principal Name,Email,Ethos Description,Female,Male,Total,latitude,longitude,Type
0,00697S,ST BRIGIDS MXD N S,18214040,Denis Courtney,principal@saintbrigids.ie,Catholic,447,502.0,949,53.375855,-6.361625,Primary School
1,00714P,LUCAN B N S,16281857,Dara Burke,info@stmarysbnslucan.com,Catholic,0,513.0,513,53.360019,-6.438876,Primary School
2,00729F,CLOCHAR LORETO N S,14931640,Sr. Maria Hyland,info@loretograngeroad.ie,Catholic,501,0.0,501,53.295317,-6.282539,Primary School
3,00752A,CENTRAL SENIOR MXD N S,18788103,Deirdre Gartland,modesenmix@eircom.net,Catholic,133,143.0,276,53.349314,-6.257670,Primary School
4,01170G,S N NA H-AILLE,18413821,Edel McMahon,naulns@eircom.net,Catholic,67,72.0,139,53.579286,-6.299281,Primary School
...,...,...,...,...,...,...,...,...,...,...,...,...
447,20520O,Dublin 6 Clonskeagh and Dublin 6W Primary School,863471806,Cian Gleeson,principal.haroldscrossetns@gmail.com,Multi Denominational,6,6.0,12,53.323932,-6.278171,Primary School
448,20523U,Goatstown Stillorgan Primary School,873878592,Diane Devereux,info@goatstownstillorganetns.com,Multi Denominational,2,6.0,8,53.278506,-6.209786,Primary School
449,20525B,Killester Raheny Clontarf Educate Together Nat...,868550965,Níamh O'Toole,info.krcetns@gmail.com,Multi Denominational,3,4.0,7,NaN,NaN,Primary School
450,20528H,River Valley CNS,858705523,Anne McArdle,info@rivervalleycns.ie,Multi Denominational,9,6.0,15,53.440622,-6.234239,Primary School


In [290]:
#pip install folium

In [291]:
#fill values that are empty
#df_new = df_new.fillna(1)

In [292]:
df_new.shape

(452, 12)

## * Working with second dataset

In [331]:
df2 = pd.read_csv('data/post-primary-schools.csv')

In [332]:
df2.head(50)

,Roll Number,Official School Name,Address 1,Address 2,Address 3,Address 4,County,Local Authority,Eircode,Principal Name,...,Island Location (Y/N),Fee Paying School (Y/N),Pupil Attendance Type,School Gender - Post Primary,Irish Classification - Post Primary,Post Primary School Type,Ethos/Religion,FEMALE,MALE,TOTAL (2019-20)
0,60010P,Loreto Secondary School,Brick Lane,Balbriggan,Co Dublin,NaN,Dublin,Fingal County Council,K32R248,MS. ANN M MCDONOUGH,...,N,N,Day,Girls,No subjects taught through Irish,Secondary,CATHOLIC,"1,282",0,"1,282"
1,60021U,St Marys Secondary School,Baldoyle,Baldoyle,NaN,NaN,Dublin,Fingal County Council,D13W208,MS. EDEL GREENE,...,N,N,Day,Girls,No subjects taught through Irish,Secondary,CATHOLIC,274,0,274
2,60030V,Blackrock College,Blackrock College,Blackrock,NaN,NaN,Dublin,Dun Laoghaire Rathdown,A94FK84,MR. ALAN T MAC GINTY,...,N,Y,Mixed,Boys,No subjects taught through Irish,Secondary,CATHOLIC,0,"1,024","1,024"
3,60040B,Willow Park School,Rock Road,Blackrock,Co Dublin,NaN,Dublin,Dun Laoghaire Rathdown,A94TW98,MR. ALAN JAMES THOMAS ROGAN,...,N,Y,Mixed,Boys,No subjects taught through Irish,Secondary,CATHOLIC,0,207,207
4,60041D,Coláiste Eoin,Baile an Bhóthair,Bóthair Stigh Lorgan,Co. Átha Cliath,NaN,Dublin,Dun Laoghaire Rathdown,A94E122,MR. P.S DE POIRE,...,N,N,Day,Boys,All pupils taught all subjects through Irish,Secondary,CATHOLIC,0,496,496
5,60042F,Coláiste Íosagáin,Bóthar Stigh Lorgan,Baile an Bhóthair,An Charraig Dhubh,Co Átha Cliath,Dublin,Dun Laoghaire Rathdown,A94KV12,MR. SEAN DELAP,...,N,N,Day,Girls,All pupils taught all subjects through Irish,Secondary,CATHOLIC,485,0,485
6,60050E,Oatlands College,Mount Merrion,Co Dublin,NaN,NaN,Dublin,Dun Laoghaire Rathdown,A94HX38,MS. CAROLINE GARRETT,...,N,N,Day,Boys,No subjects taught through Irish,Secondary,CATHOLIC,0,545,545
7,60070K,Dominican College,Dominican College Sion Hill,Cross Avenue,Blackrock,Co Dublin,Dublin,Dun Laoghaire Rathdown,A94TP97,MISS SHEILA DRUM,...,N,N,Day,Girls,No subjects taught through Irish,Secondary,CATHOLIC,437,0,437
8,60081P,Rockford Manor Secondary School,Stradbrook Rd,Blackrock,Co Dublin,NaN,Dublin,Dun Laoghaire Rathdown,A94H294,MR. JOHN (TONY) COLLISON,...,N,N,Day,Girls,No subjects taught through Irish,Secondary,CATHOLIC,277,0,277
9,60090Q,Rathdown School,Upper Glenageary Road,Glenageary,Co. Dublin,NaN,Dublin,Dun Laoghaire Rathdown,A96NX44,MR. BRIAN MOORE,...,N,Y,Mixed,Girls,No subjects taught through Irish,Secondary,CHURCH OF IRELAND,263,0,263


In [333]:
df2.shape

(183, 24)

In [334]:
df2 = df2.rename({'Address 1':'address1', 'Address 2':'address2', 'Address 3':'address3', 'Address 4':'address4', 'Official School Name': 'schoolName', 'TOTAL (2019-20)': 'Total', 'Ethos/Religion': 'Ethos Description', 'Phone No.': 'Phone'}, axis='columns')

In [335]:
df2['address'] = df2.address1 + "," + df2.address2 + "," + df2.County

In [336]:
df2['gcode'] = df2.address.apply(locator.geocode)

In [337]:
from geopy.extra.rate_limiter import RateLimiter
#1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df2['location'] = df2['address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df2['point'] = df2['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df2[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df2['point'].tolist(), index=df2.index)

In [338]:
df2.head()

,Roll Number,schoolName,address1,address2,address3,address4,County,Local Authority,Eircode,Principal Name,...,FEMALE,MALE,Total,address,gcode,location,point,latitude,longitude,altitude
0,60010P,Loreto Secondary School,Brick Lane,Balbriggan,Co Dublin,NaN,Dublin,Fingal County Council,K32R248,MS. ANN M MCDONOUGH,...,"1,282",0,"1,282","Brick Lane,Balbriggan,Dublin","(Brick Lane, K32 WY80, Balbriggan, Éire / Irel...","(Brick Lane, K32 WY80, Balbriggan, Éire / Irel...","(53.6122056, -6.1865397, 0.0)",53.612206,-6.186540,0.0
1,60021U,St Marys Secondary School,Baldoyle,Baldoyle,NaN,NaN,Dublin,Fingal County Council,D13W208,MS. EDEL GREENE,...,274,0,274,"Baldoyle,Baldoyle,Dublin","(Baldoyle ED, Fingal, Éire / Ireland, (53.3997...","(Baldoyle ED, Fingal, Éire / Ireland, (53.3997...","(53.399792149999996, -6.138721275936534, 0.0)",53.399792,-6.138721,0.0
2,60030V,Blackrock College,Blackrock College,Blackrock,NaN,NaN,Dublin,Dun Laoghaire Rathdown,A94FK84,MR. ALAN T MAC GINTY,...,0,"1,024","1,024","Blackrock College,Blackrock,Dublin","(Blackrock College, Rock Road, D04 R2C5, Rock ...","(Blackrock College, Rock Road, D04 R2C5, Rock ...","(53.3047146, -6.192405228920508, 0.0)",53.304715,-6.192405,0.0
3,60040B,Willow Park School,Rock Road,Blackrock,Co Dublin,NaN,Dublin,Dun Laoghaire Rathdown,A94TW98,MR. ALAN JAMES THOMAS ROGAN,...,0,207,207,"Rock Road,Blackrock,Dublin","(Blackrock College, Rock Road, D04 R2C5, Rock ...","(Blackrock College, Rock Road, D04 R2C5, Rock ...","(53.3047146, -6.192405228920508, 0.0)",53.304715,-6.192405,0.0
4,60041D,Coláiste Eoin,Baile an Bhóthair,Bóthair Stigh Lorgan,Co. Átha Cliath,NaN,Dublin,Dun Laoghaire Rathdown,A94E122,MR. P.S DE POIRE,...,0,496,496,"Baile an Bhóthair,Bóthair Stigh Lorgan,Dublin",None,None,None,NaN,NaN,NaN


In [339]:
df2 = df2.rename({'FEMALE':'Female', 'MALE': 'Male'}, axis='columns')

In [340]:
df2['Type'] = 'Secondary School'


In [341]:
df2.shape

(183, 32)

## getting both datasets together 

In [342]:
df_new_df2 = df_new.append(df2)

In [343]:
df_new_df2

,Roll Number,schoolName,Phone No.,Principal Name,Email,Ethos Description,Female,Male,Total,latitude,...,Fee Paying School (Y/N),Pupil Attendance Type,School Gender - Post Primary,Irish Classification - Post Primary,Post Primary School Type,address,gcode,location,point,altitude
0,00697S,ST BRIGIDS MXD N S,18214040.0,Denis Courtney,principal@saintbrigids.ie,Catholic,447,502,949,53.375855,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00714P,LUCAN B N S,16281857.0,Dara Burke,info@stmarysbnslucan.com,Catholic,0,513,513,53.360019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00729F,CLOCHAR LORETO N S,14931640.0,Sr. Maria Hyland,info@loretograngeroad.ie,Catholic,501,0,501,53.295317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00752A,CENTRAL SENIOR MXD N S,18788103.0,Deirdre Gartland,modesenmix@eircom.net,Catholic,133,143,276,53.349314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01170G,S N NA H-AILLE,18413821.0,Edel McMahon,naulns@eircom.net,Catholic,67,72,139,53.579286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,91339F,Hartstown Community School,NaN,MS. LUCIA GRIFFIN,info@hartstowncs.com,INTER DENOMINATIONAL,535,595,"1,130",53.392859,...,N,Day,Mixed,No subjects taught through Irish,Community,"Hartstown,Clonsilla,Dublin","(Hartstown, Clonsilla, Éire / Ireland, (53.392...","(Hartstown, Clonsilla, Éire / Ireland, (53.392...","(53.3928586, -6.4194428, 0.0)",0.0
179,91342R,Pobalscoil Neasáin,NaN,MR. PATRICK MCKENNA,office@psn.ie,INTER DENOMINATIONAL,306,468,774,53.395830,...,N,Day,Mixed,No subjects taught through Irish,Community,"Baldoyle,Dublin 13,Dublin","(Baldoyle, Sutton, Éire / Ireland, (53.3958303...","(Baldoyle, Sutton, Éire / Ireland, (53.3958303...","(53.3958303, -6.1285497, 0.0)",0.0
180,91343T,St. Tiernan's Community School,NaN,MR. DECLAN HUGHES,runai@tiernans.ie,INTER DENOMINATIONAL,160,185,345,53.281136,...,N,Day,Mixed,No subjects taught through Irish,Community,"Parkvale,Balally,Dublin","(Parkvale, Éire / Ireland, (53.28113615, -6.23...","(Parkvale, Éire / Ireland, (53.28113615, -6.23...","(53.28113615, -6.234916153582422, 0.0)",0.0
181,91344V,Rosmini Community School,NaN,MR. DARRELL DARCY,info@rosmini.ie,INTER DENOMINATIONAL,37,83,120,NaN,...,N,Day,Mixed,No subjects taught through Irish,Community,"All Hallows College Campus DCU,Gracepark Road,...",None,None,None,NaN


## Deliting colunms that are not relevant for the project

In [344]:
# delete columns that are not relevant
df_new_df2 = df_new_df2.drop(['address1', 'address2', 'address3', 'address4', 'address', 'altitude', 'County', 'Eircode', 'DEIS (Y/N)', 'Gaeltacht Area Location (Y/N)', 'Island Location (Y/N)', 'Post Primary School Type','gcode', 'location','point', 'Local Authority', 'Fee Paying School (Y/N)', 'Pupil Attendance Type', 'School Gender - Post Primary' ,'Irish Classification - Post Primary', 'Phone','Email', 'Phone No.'], axis =1)

In [345]:
df_new_df2.head()

,Roll Number,schoolName,Principal Name,Ethos Description,Female,Male,Total,latitude,longitude,Type
0,00697S,ST BRIGIDS MXD N S,Denis Courtney,Catholic,447,502,949,53.375855,-6.361625,Primary School
1,00714P,LUCAN B N S,Dara Burke,Catholic,0,513,513,53.360019,-6.438876,Primary School
2,00729F,CLOCHAR LORETO N S,Sr. Maria Hyland,Catholic,501,0,501,53.295317,-6.282539,Primary School
3,00752A,CENTRAL SENIOR MXD N S,Deirdre Gartland,Catholic,133,143,276,53.349314,-6.257670,Primary School
4,01170G,S N NA H-AILLE,Edel McMahon,Catholic,67,72,139,53.579286,-6.299281,Primary School


In [346]:
df_new_df2 = df_new_df2.fillna(1)
#df_new_df2 = df_new_df2.dropna()

In [347]:
df_new_df2.shape

(635, 10)

In [348]:
import folium

In [349]:
# create the map.
map_main = folium.Map( location=[53.5000, -6.20000])

# adding the latitude and longitude points to the map.
df_new_df2.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]] ).add_to(map_main), axis=1)

# display the map: just ask for the object representation in juypter notebook.
map_main

# optional: save the map.
map_main.save('map_main.html')